# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Pipeline</span>

<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group, create training dataset within the feature store, train a model and save it to model registry.</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups.
2. Define Transformation functions.
3. Create Feature Views.
4. Create Training Dataset with training, validation and test splits.
5. Train the model.
6. Register model in Hopsworks Model Registry.

![part2](../../images/02_training-dataset.png) 

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U xgboost --quiet

In [ ]:
import joblib
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import time

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

### <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

You will start by selecting all the features you want to include for model training/inference.

In [ ]:
# Retrieve feature groups
trans_fg = fs.get_feature_group(
    name='transactions_fraud_streaming_fg', 
    version=1,
)
window_aggs_fg = fs.get_feature_group(
    name='transactions_aggs_fraud_streaming_fg', 
    version=1,
)

In [ ]:
# Select features for training data.
selected_features = trans_fg.select(["fraud_label", "category", "amount", "datetime", "age_at_transaction", "days_until_card_expires"])\
    .join(window_aggs_fg.select_except(["cc_num", "datetime"]))

In [ ]:
# Uncomment this if you would like to view your selected features
#selected_features.read()

Recall that you computed the features in `transactions_4h_aggs_fraud_batch_fg` using 4-hour aggregates. If you had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join you would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

---

### <span style="color:#ff5f27;"> 🤖 Transformation Functions </span>


You will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this you simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [ ]:
# Load transformation functions.
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = [
    label_encoder("category") ,
]

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View you may use `fs.create_feature_view()`. Here we try first to get the feature view, and if we can't an exception is thrown and we create the feature view.

In [ ]:
# Get or create the 'transactions_view_fraud_batch_fv' feature view
feature_view = fs.get_or_create_feature_view(
    name='transactions_view_streaming_fv',
    version=1,
    query=selected_features,
    labels=["fraud_label"],
    transformation_functions=transformation_functions,
)

The feature view is now visible in the UI.

![fg-overview](../../images/fv_overview.gif)

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `feature_view.train_validation_test_split()` method.

In [ ]:
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size = TEST_SIZE
)

In [ ]:
# Sort the X_train DataFrame based on the "datetime" column in ascending order
X_train = X_train.sort_values("datetime")

# Reindex the y_train Series to match the order of rows in the sorted X_train DataFrame
y_train = y_train.reindex(X_train.index)

In [ ]:
# Sort the X_test DataFrame based on the "datetime" column in ascending order
X_test = X_test.sort_values("datetime")

# Reindex the y_test Series to match the order of rows in the sorted X_test DataFrame
y_test = y_test.reindex(X_test.index)

In [ ]:
# Drop the "datetime" column from the X_train DataFrame along the specified axis (axis=1 means columns)
X_train.drop(["datetime"], axis=1, inplace=True)

# Drop the "datetime" column from the X_test DataFrame along the specified axis (axis=1 means columns)
X_test.drop(["datetime"], axis=1, inplace=True)

In [ ]:
# Display the normalized value counts of the y_train Series
y_train.value_counts(normalize=True)

Notice that the distribution is extremely skewed, which is natural considering that fraudulent transactions make up a tiny part of all transactions. Thus you should somehow address the class imbalance. There are many approaches for this, such as weighting the loss function, over- or undersampling, creating synthetic data, or modifying the decision threshold. In this example, you will use the simplest method which is to just supply a class weight parameter to our learning algorithm. The class weight will affect how much importance is attached to each class, which in our case means that higher importance will be placed on positive (fraudulent) samples.

---

## <span style="color:#ff5f27;"> 🧬 Modeling</span>

Next you will train a model. Here, you set larger class weight for the positive class.

In [ ]:
# Create an instance of the XGBClassifier
clf = xgb.XGBClassifier()

# Fit the classifier on the training data
clf.fit(X_train.values, y_train)

In [ ]:
# Predict the training data using the trained classifier
y_pred_train = clf.predict(X_train.values)

# Predict the test data using the trained classifier
y_pred_test = clf.predict(X_test.values)

In [ ]:
# Compute f1 score
metrics = {
    "f1_score": f1_score(y_test, y_pred_test, average='macro')
}
metrics

In [ ]:
# Generate the confusion matrix using the true labels (y_test) and predicted labels (y_pred_test)
results = confusion_matrix(y_test, y_pred_test)

# Print the confusion matrix
print(results)

In [ ]:
# Create a DataFrame from the confusion matrix results with appropriate labels
df_cm = pd.DataFrame(
    results, 
    ['True Normal', 'True Fraud'],
    ['Pred Normal', 'Pred Fraud'],
)

# Create a heatmap using seaborn with annotations
cm = sns.heatmap(df_cm, annot=True)

# Get the figure from the heatmap and display it
fig = cm.get_figure()
fig.show()

---

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/3.0/user_guides/mlops/registry/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define the input schema using the values of X_train
input_schema = Schema(X_train.values)

# Define the output schema using y_train
output_schema = Schema(y_train)

# Create a ModelSchema object specifying the input and output schemas
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Convert the model schema to a dictionary for further inspection or serialization
model_schema.to_dict()

## <span style="color:#ff5f27;">📝 Register model</span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
# Specify the directory where the model will be saved
model_dir = "fraud_streaming_model"

# Check if the directory exists, and create it if it doesn't
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Save the trained XGBoost model using joblib
joblib.dump(clf, model_dir + '/xgboost_fraud_streaming_model.pkl')

# Save the confusion matrix heatmap as an image in the model directory
fig.savefig(model_dir + "/confusion_matrix.png")

In [ ]:
# Get the model registry
mr = project.get_model_registry()

# Create a new model in the model registry
fraud_model = mr.python.create_model(
    name="xgboost_fraud_streaming_model",     # Name for the model
    metrics=metrics,                      # Metrics used for evaluation
    model_schema=model_schema,            # Schema defining the model's input and output
    input_example=X_train.sample(),       # Example input data for reference
    description="Fraud Batch Predictor",  # Description of the model
)

# Save the model to the specified directory
fraud_model.save(model_dir)

## <a class="anchor" id="1.5_bullet" style="color:#ff5f27"> 🚀 Model Deployment</a>


### About Model Serving
Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, you do not need to write a prediction file (see the section below). However, for sklearn models using default serving, you do need to proceed to write a prediction file.

In order to use KFServing, you must have Kubernetes installed and enabled on your cluster.

### <span style="color:#ff5f27;">📎 Predictor script for Python models</span>


Scikit-learn and XGBoost models are deployed as Python models, in which case you need to provide a **Predict** class that implements the **predict** method. The **predict()** method invokes the model on the inputs and returns the prediction as a list.

The **init()** method is run when the predictor is loaded into memory, loading the model from the local directory it is materialized to, *ARTIFACT_FILES_PATH*.

The directive "%%writefile" writes out the cell before to the given Python file. We will use the **predict_example.py** file to create a deployment for our model. 

In [ ]:
%%writefile predict_example.py
import os
import numpy as np
import hsfs
import joblib


class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # Get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # Get feature view
        self.fv = self.fs.get_feature_view("transactions_view_streaming_fv", 1)
        
        # Initialize serving
        self.fv.init_serving(1)

        # Load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/xgboost_fraud_streaming_model.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        feature_vector = self.fv.get_feature_vector({"cc_num": inputs[0][0]}, return_type="pandas").drop(["datetime"], axis=1).values
        return self.model.predict(feature_vector.reshape(1, -1)).tolist() # Numpy Arrays are not JSON serializable

If you wonder why we use the path Models/fraud_tutorial_model/1/model.pkl, it is useful to know that the Data Sets tab in the Hopsworks UI lets you browse among the different files in the project. Registered models will be found underneath the Models directory. Since you saved you model with the name fraud_tutorial_model, that's the directory you should look in. 1 is just the version of the model you want to deploy.

This script needs to be put into a known location in the Hopsworks file system. Let's call the file predict_example.py and put it in the Models directory.

In [ ]:
# Get the dataset API for the current project
dataset_api = project.get_dataset_api()

# Specify the local file path of the Python script to be uploaded
local_script_path = "predict_example.py"

# Upload the Python script to the "Models", and overwrite if it already exists
uploaded_file_path = dataset_api.upload(local_script_path, "Models", overwrite=True)

# Create the full path to the uploaded script for future reference
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

### Create the deployment
Here, you fetch the model you want from the model registry and define a configuration for the deployment. For the configuration, you need to specify the serving type (default or KFserving).

In [ ]:
# Deploy the fraud model
deployment = fraud_model.deploy(
    name="fraudonlinemodeldeployment",  # Specify a name for the deployment
    script_file=predictor_script_path,  # Provide the path to the Python script for prediction
)

In [ ]:
# Print the name of the deployment
print("Deployment: " + deployment.name)

# Display information about the deployment
deployment.describe()

In [ ]:
print("Deployment is warming up...")
time.sleep(45)

#### The deployment has now been registered. However, to start it you need to run the following command:

In [ ]:
# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

In [ ]:
# Get the current state of the deployment
deployment.get_state().describe()

In [ ]:
# To troubleshoot you can use `get_logs()` method
deployment.get_logs(component='predictor')

### Stop Deployment
To stop the deployment you simply run:

In [ ]:
# Stop the deployment and wait for it to be in a stopped state for up to 180 seconds
deployment.stop(await_stopped=180)

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Online Inference</span>

In the following notebook you will use your model for online inference.
